In [1]:
!pip install keras_cv_attention_models

  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=178f232c3a7c7595985994f409bb0b01e2427c2ff8f173102322c53afc5f30c9
  Stored in directory: c:\users\pnu\appdata\local\pip\cache\wheels\e1\e8\83\ddea66100678d139b14bc87692ece57c6a2a937956d2532608
Successfully built promise


In [1]:
# 모델 불러오기 
import tensorflow as tf
import keras 
from keras_cv_attention_models import mobilevit

mm = keras.models.load_model('first_model.h5')

# 제너레이터 쓴 버전이랑 안쓴 버전 둘다 테스트 해주세요|

In [2]:
from tensorflow.keras import layers
resize_and_rescale = tf.keras.Sequential([
  
  layers.experimental.preprocessing.Rescaling(1./255)
])

mm = tf.keras.Sequential([
    layers.Input(shape=(256, 256, 3)),
  layers.experimental.preprocessing.Rescaling(1./255),
    layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.2),
  mm
])
mm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 random_flip (RandomFlip)    (None, 256, 256, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 256, 256, 3)      0         
 tion)                                                           
                                                                 
 mobilevit_s (Functional)    (None, 7)                 4954375   
                                                                 
Total params: 4,954,375
Trainable params: 4,942,119
Non-trainable params: 12,256
_________________________________________________________________


In [ ]:
# 제너레이터 쓴 버전

import shutil
import pathlib
from tensorflow.keras.utils import image_dataset_from_directory 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_path = r'C:\workplace\holo-project\data_split\train'
val_path = r'C:\workplace\holo-project\data_split\val'
size = (256,256)
batch_size = 16
num_epochs = 50
input_shape = (256,256,3)

train_datagen = ImageDataGenerator(
                                   rescale=1./255,
#                                    rotation_range=40,
#                                    shear_range=0.5,
#                                    zoom_range=0.2,
                                   vertical_flip=True,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_ds = train_datagen.flow_from_directory(train_path,
                                                  target_size=size,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=batch_size)

valid_datagen = ImageDataGenerator(
                                  rescale=1./255)
val_ds = valid_datagen.flow_from_directory(val_path,
                                                  target_size=size,
                                                  
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=batch_size)

# show class indices
print('****************')
for cls, idx in train_ds.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
print('****************')

In [3]:
import shutil
import pathlib
import tensorflow as tf

data_dir = pathlib.Path(r'C:\BS\data')
print(len(list(data_dir.glob(r'*\*.jpg'))))

73776


In [4]:
# 데이터 생으로 쓴 버전

import shutil
import pathlib
import tensorflow as tf

data_dir = pathlib.Path(r'C:\BS\data')
print(len(list(data_dir.glob(r'*\*.jpg'))))


batch_size = 16
# num_epochs = 100
img_height = 256
img_width = 256

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = 'training',
    seed = 97,
    shuffle=True,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical')
    
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = 'validation',
    seed = 97,
    shuffle=True,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical')

class_names = train_ds.class_names
print(class_names)
print(train_ds)
print(len(train_ds))

73776
Found 73776 files belonging to 7 classes.
Using 59021 files for training.
Found 73776 files belonging to 7 classes.
Using 14755 files for validation.
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))>
3689


In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
from tensorflow import keras
import tensorflow_addons as tfa
import numpy as np

label_smoothing_factor = 0.1
epochs = 100
num_classes = 7
initial_learning_rate=0.02
steps = np.round(len(train_ds)/ 32)


# 처음에는 learning rate는 0.002 / 배치사이즈는 할 수 있는 만큼 크게 해서 돌려주세요


learning_rate = 0.0002  # 논문에서는 0.002에서 시작  


optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss_fn =  keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing_factor)


reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, # 저는 0.1 or 0.5로 조정하면서 했습니다. 함수 설명 보시고 커스텀 해주세요
    patience=2, # 이것도 학습되는거 보시고 적절하게 바꾸시면 될거같습니다./ train에서 과적합이 너무 많이 뜨면 1~2, 괜찮으면 좀 더 높게
    min_lr = 1e-8 
    )    



def run_experiment(epochs=epochs):
    
    mm.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 4)
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath = 'mobileVIT_{epoch:02d}-{val_loss:.2f}.h5',
    monitor = 'val_loss',
    save_best_only = True, verbose = 1,
    )

    mm.fit(
        train_ds,
        validation_data = val_ds,
        epochs=epochs,
        batch_size = 16, # 배치사이즈 
        callbacks = [early_stopping, model_checkpoint, reduce_lr],
    )
    
    _, accuracy = mm.evaluate(val_ds)
    print(f"Validation accuracy: {round(accuracy * 100, 2)}%")
    return mm


model = run_experiment()

Epoch 1/100
3689/3689 [==============================] - ETA: 0s - loss: 1.2453 - accuracy: 0.6078
Epoch 1: val_loss improved from inf to 1.07114, saving model to mobileVIT_01-1.07.h5
3689/3689 [==============================] - 1530s 411ms/step - loss: 1.2453 - accuracy: 0.6078 - val_loss: 1.0711 - val_accuracy: 0.6924 - lr: 2.0000e-04
Epoch 2/100
3689/3689 [==============================] - ETA: 0s - loss: 1.0650 - accuracy: 0.7003
Epoch 2: val_loss improved from 1.07114 to 1.01171, saving model to mobileVIT_02-1.01.h5
3689/3689 [==============================] - 1519s 412ms/step - loss: 1.0650 - accuracy: 0.7003 - val_loss: 1.0117 - val_accuracy: 0.7231 - lr: 2.0000e-04
Epoch 3/100
3689/3689 [==============================] - ETA: 0s - loss: 0.9909 - accuracy: 0.7368
Epoch 3: val_loss did not improve from 1.01171
3689/3689 [==============================] - 1519s 412ms/step - loss: 0.9909 - accuracy: 0.7368 - val_loss: 1.0237 - val_accuracy: 0.7182 - lr: 2.0000e-04
Epoch 4/100
3689/

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

In [3]:
# 모델 불러오기 
import tensorflow as tf
import keras 
from keras_cv_attention_models import mobilevit

mm = keras.models.load_model('mobile_VIT.h5')

# 제너레이터 쓴 버전이랑 안쓴 버전 둘다 테스트 해주세요|

In [7]:
!python -m tf2onnx.convert --opset 13 --saved-model 'mobile_VIT.h5' --output 'mobile_VIT.onnx'

C:\Users\pusan\.conda\envs\gibo-project\lib\runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-05-24 03:03:56,333 - WARNING - '--tag' not specified for saved_model. Using --tag serve
Traceback (most recent call last):
  File "C:\Users\pusan\.conda\envs\gibo-project\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\pusan\.conda\envs\gibo-project\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\pusan\.conda\envs\gibo-project\lib\site-packages\tf2onnx\convert.py", line 640, in <module>
    main()
  File "C:\Users\pusan\.conda\envs\gibo-project\lib\site-packages\tf2onnx\convert.py", line 235, in main
    graph_def, inputs, outputs, initialized_tables, tensors_to_rename = tf_loader.from_saved_model(
  File "C:\Users\pusan\.con

In [8]:
!python -m tf2onnx.convert --saved-model mobile_VIT.h5 --opset 13 --output mobile_VIT.h5.onnx

C:\Users\pusan\.conda\envs\gibo-project\lib\runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-05-24 03:10:19,635 - WARNING - '--tag' not specified for saved_model. Using --tag serve
Traceback (most recent call last):
  File "C:\Users\pusan\.conda\envs\gibo-project\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\pusan\.conda\envs\gibo-project\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\pusan\.conda\envs\gibo-project\lib\site-packages\tf2onnx\convert.py", line 640, in <module>
    main()
  File "C:\Users\pusan\.conda\envs\gibo-project\lib\site-packages\tf2onnx\convert.py", line 235, in main
    graph_def, inputs, outputs, initialized_tables, tensors_to_rename = tf_loader.from_saved_model(
  File "C:\Users\pusan\.con

In [11]:
from tensorflow import keras
mm = keras.models.load_model('mobile_VIT.h5')



mm.save('C:\workplace\zmq', save_format='tf')

INFO:tensorflow:Assets written to: C:\workplace\zmq\assets


INFO:tensorflow:Assets written to: C:\workplace\zmq\assets
